beta -> Rate at which people with virus are coming in contact with suceptible people \
beta1 -> Lockdown leak rate \
b1 -> Asymptomatic interaction rate with suceptible \
b2 -> Symtomatic interation rate with suceptible \
b3 -> Patients interaction with suceptible \
k0 -> Suceptible going lockdown rate \
mu -> Suceptible leaving lockdown rate \
sigma -> Asymptomatic becomes symptomatic \
kt -> Testing rate \
gamma1 -> Asymptomatic removal rate \
gamma2 -> Infected removal rate \
gamma3 -> Patient removal rate \

Source for gammas: https://www.who.int/docs/default-source/coronaviruse/who-china-joint-mission-on-covid-19-final-report.pdf \
\
From page 14(first paragraph): \
Using available preliminary data, the median time from onset to clinical recovery for mild \
cases is approximately 2 weeks and is 3-6 weeks for patients with severe or critical disease. \
Preliminary data suggests that the time period from onset to the development of severe \
disease, including hypoxia, is 1 week. Among patients who have died, the time from \
symptom onset to outcome ranges from 2-8 weeks.\
\
gamma1=(1/14)=gamma2 \
gamma3=(1/28) (four weeks) \

Total number of testing data is taken from https://www.worldometers.info/coronavirus/covid-19-testing/ \
feb26 = 9462\
mar2 = 23345\
mar9= 60761\
I am assuming the rate of testing between mar2 to mar9 i.e. 5345 tests per day is maintained daily in future \
kt=5345

Source of sigma https://www.healthline.com/health/coronavirus-incubation-period \
It takes five days for symptoms to show \
sigma = 0.2

Lockdown was declared on 9th March, till 3rd April  94% drop for retail and recreation places, an 85% drop for grocery shops and pharmacies, a 90% drop for parks and beaches, an 87% drop for public transit hubs, a 63% drop for workplaces, and a 24% rise for residential places. \
k0=1/25 \
Source: https://en.wikipedia.org/wiki/2020_Italy_coronavirus_lockdown#Effectiveness

mu=1/7     Guess \
b1=0.1     Guess \
b2=0.01    Guess \
b3=0.001   Guess 


Lockdown in Italy started in march 9th data available from feb 15th \
So, Lockdown starts from day 23 \
Total population of italy 60482635


In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import importlib
import pandas as pd

In [2]:
data=pd.read_csv("Italy_Data.csv")
dates=data['Date'][8:].values
actualI=data['Active Cases in Italy'].values[8:]
actualDeaths=data["Total Coronavirus Deaths in Italy"].values[8:]
#dates=(dates[:15])
#actualI=actualI[:15]
time=list(range(len(dates)))

In [6]:
def getPreds(x,beta,beta1):
    
    N=60482635  #population
    gamma1=1/19
    gamma2=1/14
    gamma3=1/14
    kt=0.06125
    sigma = 0.1
    k0=1/7
    mu=1/7
    b1=beta1
    b2=0.1*beta1
    b3=0.001*beta1
    beta=0.
    lkdwn_start="24"
    lkdwn_end="50"
    
    t=np.linspace(0,40,401)
    
    def sirx(x,t):
        #import the values
        S,A,I,Xs,Xa,Xi,P,R=x

        lkdwn_start1=list(map(int,lkdwn_start.split(',')))
        lkdwn_end1=list(map(int,lkdwn_end.split(',')))
        lkdwn_start1.append(days)

        #For heavyside function
        for j in range(len(lkdwn_end1)):
            if t>lkdwn_end1[j] and t<lkdwn_start1[j+1]:
                theta=1
                break
            else:
                theta=0
        #For Pulse function
        for j in range(len(lkdwn_end1)):
            if t>lkdwn_start1[j] and t<lkdwn_end1[j]:
                zi=1
                break
            else:
                zi=0

        com=beta*(I+A+(b1*Xa)+(b2*Xi)+(b3*P))

                        #THE EQUATIONS
        dSdt = -(com*S/N) - (zi*k0*S) + (theta*mu*Xs)
        dAdt = (com*S/N) - (sigma*A) - (zi*k0*A) + (theta*mu*Xa) - (gamma1*A)
        dIdt = (sigma*A) - (kt*I) - (zi*k0*I) + (theta*mu*Xi) - (gamma2*I)                              
        dXsdt = -(beta1*com*Xs) + (zi*k0*S) - (theta*mu*Xs)
        dXadt = (beta1*com*Xs) - (sigma*Xa) + (zi*k0*A) - (theta*mu*Xa) - (gamma1*Xa)
        dXidt = (sigma*Xa) - (kt*Xi) + (zi*k0*I) - (theta*mu*Xi) - (gamma2*Xi)
        dPdt = (kt*(I+Xi)) - (gamma3*P)
        dRdt = (gamma1*(A+Xa)) + (gamma2*(I+Xi)) + (gamma3*P)

        return [dSdt,dAdt,dIdt,dXsdt,dXadt,dXidt,dPdt,dRdt]
    
    ret = odeint(sirx, x, t)
    
    S,A,I,Xs,Xa,Xi,P,R = ret.T
    
    return P

In [7]:
days=15 #No. of days of simulation

N=60461826            # Total population
I=912              # Infected 
A=4256            # Asymptomatic
Xs=0             # Quarantined Susecptible
Xa=0              # Quarantined Asymptomatic 
Xi=0              # Quarantined Infected
P=3             #Patients
R=0             #Removed
S= N - A - I - Xs - Xa -Xi - P - R # Suceptible people

x0=[S,A,I,Xs,Xa,Xi,P,R]

In [8]:
# Define loss and search for the best hyperparameters (grid search for now)
def squaredLoss(preds, target):
    
    loss = ((preds - target) ** 2).sum()
    return loss

In [9]:
bestLoss = 10000000000000000000000000
bestBeta = -1, -1
bestPreds = None
betaValues=np.linspace(0.26,0.28,21)
beta1Values = np.linspace(0., 1., 101)
samplesPerDay=10
#beta=0.34157894736842104

for beta in betaValues:
    for beta1 in beta1Values:
        predI = getPreds(x0, beta, beta1)
        trimmedPredI = predI[:len(actualI) * samplesPerDay:samplesPerDay]
        assert len(trimmedPredI) == len(actualI), "Length mismatch"
        #           loss = squaredLossExpScale(trimmedPredI, actualI) + squaredLossExpScale(trimmedPredR, actualR)
        loss = squaredLoss(trimmedPredI,actualI)

        if loss < bestLoss:
            bestLoss = loss
            bestPreds = predI
            bestBeta = beta
    print(loss,beta)
print("Best beta value", bestBeta)

68819251017.47581 0.26
68819251017.47581 0.261
68819251017.47581 0.262
68819251017.47581 0.263
68819251017.47581 0.264
68819251017.47581 0.265
68819251017.47581 0.266
68819251017.47581 0.267
68819251017.47581 0.268
68819251017.47581 0.269
68819251017.47581 0.27
68819251017.47581 0.271
68819251017.47581 0.272
68819251017.47581 0.273
68819251017.47581 0.274
68819251017.47581 0.275
68819251017.47581 0.276
68819251017.47581 0.277
68819251017.47581 0.278
68819251017.47581 0.279
68819251017.47581 0.28
Best beta value 0.26


In [17]:
print(bestLoss)

28437875.930030987
